<a href="https://colab.research.google.com/github/yash-1708/Stock-Analyst-Rating/blob/master/scrapingmarketsmojoBIG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
from dateutil import parser
import sqlalchemy
from datetime import datetime

In [2]:
driver = webdriver.Chrome("chromedriver.exe")
broker=[] #List to store name of the broker
company_name=[] #List to store company names 
current_price=[] #List to store rating of the product
predict_date=[]
recomended_buying=[]
target_price=[]
driver.get("https://www.marketsmojo.com/recommendation/")


In [3]:
content = driver.page_source
soup = BeautifulSoup(content,'html.parser')
Alldata=soup.select_one("div.righttexcontent2 div.right-table01 div.scr-result2")
# for row in Alldata.findAll('div',class_='middle-colum02'):
for row in Alldata.findAll('div',attrs={'class':'middle-colum02'}):
    broker.append(row.select("div.colum2-midd01 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text())
    company_name.append(row.select("div.colum2-midd02 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text().replace('\t','').replace('\n',''))
    current_price.append(row.select("div.colum2-midd03 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    recomended_buying.append(row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.text-top0")[0].get_text())
    target_price.append(row.select("div.colum2-midd05 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    temp_data=row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.colum-text-color-bott")
    temp_data=str(temp_data).split('on ')[1]
    temp_data=temp_data.split('<br/')[0]
    predict_date.append(temp_data)
    #predict_date.append(row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.colum-text-color-bott"))
     
for row in Alldata.findAll('div',attrs={'class':'middle-columactive'}):
    broker.append(row.select("div.colum2-midd01 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text())
    company_name.append(row.select("div.colum2-midd02 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text().replace('\t','').replace('\n',''))
    current_price.append(row.select("div.colum2-midd03 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    recomended_buying.append(row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.text-top0")[0].get_text())
    target_price.append(row.select("div.colum2-midd05 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    temp_data=row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.colum-text-color-bott")
    temp_data=str(temp_data).split('on ')[1]
    temp_data=temp_data.split('<br/')[0]
    predict_date.append(temp_data)

In [4]:
temp_data

'24 Oct 19'

In [5]:
df1 = pd.DataFrame({'broker':broker})  
df2 = pd.DataFrame({'company_name':company_name})
df3 = pd.DataFrame({'current_price':current_price})
df4 = pd.DataFrame({'recomended_buing':recomended_buying})
df5=pd.DataFrame({'target_price':target_price})
df6=pd.DataFrame({'predict_date':predict_date})

In [6]:
rb=df4['recomended_buing'].str.split('Rs.').str[1] 
cp=df3['current_price'].str.split('Rs.').str[1]
tp=df5['target_price'].str.split('Rs.').str[1]
predict_date=df6['predict_date'].str.split('10').str[0]

In [7]:
dfall=pd.DataFrame({'broker':broker, 'company_name':company_name, 'current_price':cp, 'recomended_buying':rb, 'predict_date':predict_date, 'target_price':tp})
dfallclean = dfall[dfall['predict_date']!='']
dfallclean['predict_date'] =  pd.to_datetime(dfallclean['predict_date'], format='%d %b %y')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
dfallclean

,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,Kotak Securities,Arvind,41.65,43.10,2020-02-05,57.00
1,Prabhudas Lilladher,Shaily Engg. Plastic,554.7,570.00,2020-02-05,691.00
2,Geojit Financial Services,Can Fin Homes,479.4,483.15,2020-02-05,557.00
3,Geojit Financial Services,M&M Financial,388,367.90,2020-02-05,416.00
4,Prabhudas Lilladher,Exide Inds,186.35,188.45,2020-02-05,214.00
...,...,...,...,...,...,...
2142,Prabhudas Lilladher,M&M Financial,388,333.25,2019-10-24,362.00
2143,Prabhudas Lilladher,Rallis India,236.75,163.50,2019-10-24,174.00
2144,Prabhudas Lilladher,CEAT,984,969.15,2019-10-24,827.00
2145,Prabhudas Lilladher,Inox Leisure,411.95,354.20,2019-10-24,416.00


In [9]:
database_username = 'smarsproddbuser@mbt-smars-mysql'
database_password = 'modern@1234'
database_ip       = 'mbt-smars-mysql.mysql.database.azure.com'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [10]:
maxdate = pd.read_sql('SELECT MAX(predict_date) FROM marketsmojorecos', con=database_connection)

In [11]:
maxdate

,MAX(predict_date)
0,2019-12-15


In [12]:
a=maxdate.loc[0,'MAX(predict_date)']

In [13]:
date_object1=maxdate['MAX(predict_date)']
#date_object1.str
mynew=str(a)

In [14]:
date_object1 = datetime.strptime(mynew, '%Y-%m-%d').date()

In [15]:
mask = (dfallclean['predict_date'] > date_object1) 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


In [16]:
dfentry = dfallclean.loc[mask]
dfentry

,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,Kotak Securities,Arvind,41.65,43.10,2020-02-05,57.00
1,Prabhudas Lilladher,Shaily Engg. Plastic,554.7,570.00,2020-02-05,691.00
2,Geojit Financial Services,Can Fin Homes,479.4,483.15,2020-02-05,557.00
3,Geojit Financial Services,M&M Financial,388,367.90,2020-02-05,416.00
4,Prabhudas Lilladher,Exide Inds,186.35,188.45,2020-02-05,214.00
...,...,...,...,...,...,...
1681,Sharekhan,Hindustan Unilever,2160.15,1922.88,2019-12-18,2185.00
1682,Geojit Financial Services,BPCL,501.65,496.00,2019-12-18,474.00
1683,Prabhudas Lilladher,HDFC,2391.45,2430.80,2019-12-18,2568.00
1684,ICICIdirect.com,Tech Mahindra,814.05,777.44,2019-12-17,900.00


In [18]:

dfentry.to_sql(con=database_connection, name='marketsmojorecos', if_exists='append')